In [1]:
import os
import argparse

import polars as pl
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
!pwd

/Users/yiyichen/PycharmProjects/MRL/preprocessor


In [3]:
os.chdir("/Users/yiyichen/PycharmProjects/MRL/")

In [4]:
!pwd

/Users/yiyichen/PycharmProjects/MRL


In [5]:
def split_data(df):
    train, test_val = train_test_split(df, test_size=0.2, random_state=1, shuffle=True)
    val, test = train_test_split(test_val, test_size=0.5,random_state=1, shuffle=True)
    return train, val, test

# Hatespeech dataset
- not adequate, do not use this method.

In [39]:
dataset_path = "datasets/sentiment_analysis/hu/"
save_dir = os.path.join("datasets/hate_speech_detection/hungarian", "preprocessed")
datapath = os.path.join(dataset_path, "OpinHuBank_20130106.csv")
df = pd.read_csv(datapath,  encoding = "ISO-8859-2")
print(df.head())
lexica = pd.read_csv(os.path.join(dataset_path, "hurtlex_HU.tsv"), sep="\t")

   ID  START  LEN           Entity  \
0   1     28    2      Schmitt Pál   
1   2      7    2  Matolcsy György   
2   3     15    2  Szijjártó Péter   
3   4     18    2     Szabó Attila   
4   5     12    2        P. Zoltán   

                                            Sentence  \
0  A magyarok tiszta, egyértelmű, vállalható, jól...   
1  A szóvivő megerősítette: a kabinet Matolcsy Gy...   
2  A nyugdíjasok utazási kedvezményének megvonása...   
3  Itt találta el az a lövés, amellyel egy vadász...   
4  Egy csempész ismerőse szintén ,,cigiben utazó"...   

                                                 URL  Annot1  Annot2  Annot3  \
0  http://kecskemetitv.hu/hun/sa_kecskemet/i_belf...       0       0       0   
1  http://mti.hu/article-proxy/2011/02/18/szijjar...       0       0       0   
2  http://mti.hu/article-proxy/2011/02/18/szijjar...       0       0       0   
3  http://kecskemetitv.hu/hun/sa_zuschlag_per_kec...       0       0       0   
4  http://atv.hu/bunugy/20110223_f

In [40]:
lemmas = lexica.lemma.tolist()

In [41]:
df['label'] = df[["Annot1", "Annot2", "Annot3", "Annot4", "Annot5"]].mode(axis=1)[0]

In [42]:
df.label.value_counts()

 0.0    7419
-1.0    1693
 1.0     894
Name: label, dtype: int64

In [43]:
def get_hs(df):
    df.loc[(df['label']==-1) & (df["Sentence"].str.contains("|".join(lemmas))), "hsd"]= 1 # offensive and hatespeech
    df.loc[df['label']>=0, "hsd"]=0 # normal
    print(df.hsd.value_counts())
    # df.to_csv(os.path.join(sv_dir, f"{dataset}.csv"), index=False)
    return df

In [44]:
df = get_hs(df)

0.0    8313
1.0    1279
Name: hsd, dtype: int64


In [46]:
df.drop_duplicates(subset=["Sentence"], inplace=True)

In [47]:
df.hsd.value_counts()

0.0    6815
1.0     993
Name: hsd, dtype: int64

In [48]:
df= df.dropna(subset=['hsd'])

In [49]:
df[df.hsd==1].sample(1).Sentence

9938    A titkos voksolás után az államfő újságírók el...
Name: Sentence, dtype: object

In [50]:
df = df[["Sentence", "hsd"]]

In [51]:
df.drop_duplicates(subset=["Sentence"], inplace=True)

In [52]:
df.hsd.value_counts()

0.0    6815
1.0     993
Name: hsd, dtype: int64

In [53]:
df.rename(columns= {"Sentence":"text", "hsd":"label"}, inplace=True)

In [54]:
df = df.groupby("label").sample(993)
df = shuffle(df)

In [56]:
df.label.value_counts()

0.0    993
1.0    993
Name: label, dtype: int64

In [57]:
train, val, test = split_data(df)


In [59]:
len(train), len(val), len(test)

(1588, 199, 199)

In [58]:
train.to_csv(os.path.join(save_dir, "train.csv"))
val.to_csv(os.path.join(save_dir, "val.csv"))
test.to_csv(os.path.join(save_dir, "test.csv"))

# Sentiment analysis dataset

In [6]:
dataset_path = "datasets/sentiment_analysis/hu/"
save_dir = os.path.join(dataset_path, "preprocessed")
datapath = os.path.join(dataset_path, "OpinHuBank_20130106.csv")
df = pd.read_csv(datapath,  encoding = "ISO-8859-2")
print(df.head())

   ID  START  LEN           Entity  \
0   1     28    2      Schmitt Pál   
1   2      7    2  Matolcsy György   
2   3     15    2  Szijjártó Péter   
3   4     18    2     Szabó Attila   
4   5     12    2        P. Zoltán   

                                            Sentence  \
0  A magyarok tiszta, egyértelmű, vállalható, jól...   
1  A szóvivő megerősítette: a kabinet Matolcsy Gy...   
2  A nyugdíjasok utazási kedvezményének megvonása...   
3  Itt találta el az a lövés, amellyel egy vadász...   
4  Egy csempész ismerőse szintén ,,cigiben utazó"...   

                                                 URL  Annot1  Annot2  Annot3  \
0  http://kecskemetitv.hu/hun/sa_kecskemet/i_belf...       0       0       0   
1  http://mti.hu/article-proxy/2011/02/18/szijjar...       0       0       0   
2  http://mti.hu/article-proxy/2011/02/18/szijjar...       0       0       0   
3  http://kecskemetitv.hu/hun/sa_zuschlag_per_kec...       0       0       0   
4  http://atv.hu/bunugy/20110223_f

In [7]:
len(df)

10006

In [8]:
df_4 = df[df[["Annot1", "Annot2", "Annot3", "Annot4", "Annot5"]].apply(pd.Series.nunique, axis=1)==2]

In [9]:
df_5 = df[df[["Annot1", "Annot2", "Annot3", "Annot4", "Annot5"]].apply(pd.Series.nunique, axis=1)==1]

In [10]:
len(df_4)

2893

In [11]:
first_row = df.iloc[0]

In [12]:
first_row

ID                                                          1
START                                                      28
LEN                                                         2
Entity                                            Schmitt Pál
Sentence    A magyarok tiszta, egyértelmű, vállalható, jól...
URL         http://kecskemetitv.hu/hun/sa_kecskemet/i_belf...
Annot1                                                      0
Annot2                                                      0
Annot3                                                      0
Annot4                                                      0
Annot5                                                      0
Name: 0, dtype: object

In [13]:
df['label'] = df[["Annot1", "Annot2", "Annot3", "Annot4", "Annot5"]].mode(axis=1)[0]

In [14]:
df['label'].replace(to_replace=[-1,0,1], value=[0,1,2], inplace=True)


In [15]:
df['label'].value_counts()

1.0    7419
0.0    1693
2.0     894
Name: label, dtype: int64

In [18]:
len(df)

10006

In [19]:
df = df.drop_duplicates(subset=["Sentence"])

In [20]:
len(df)

8145

In [21]:
df.label.value_counts()

1.0    6179
0.0    1330
2.0     636
Name: label, dtype: int64

In [22]:
df_pos = df[df['label']==2]

In [23]:
df_neg = df[df['label']==0].sample(636)

In [24]:
df_neu = df[df['label']==1].sample(636)

In [25]:
len(df_pos)+len(df_neg)+len(df_neu)

1908

In [27]:
df_concat = pd.concat([df_pos,df_neg, df_neu])

In [28]:
df_new = df_concat[['Sentence', 'label']]

In [30]:
df_new.rename(columns={"Sentence":"text"}, inplace=True)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [31]:
#from sklearn.model_selection import train_test_split

In [67]:
# train, test_dev= train_test_split(df_new, test_size=0.39)

In [68]:
# dev, test = train_test_split(test_dev, test_size=0.27)

In [32]:
df_new = shuffle(df_new)

In [33]:
train, dev, test = split_data(df_new)

In [34]:
len(train), len(dev), len(test)

(1526, 191, 191)

In [35]:
train.head()

,text,label
2157,FOTÓ: LÉNÁRT MÁRTON Bardi Gáborék továbbra is ...,1.0
9320,Gloria Cainről az elnökaspiráns korábban azt á...,2.0
6158,"Ante Gotovina, aki jelenleg börtönbüntetését t...",0.0
9155,Karácsony Sándor-díjat kapott Varjúné Horváth ...,2.0
968,"Farkas Béla, a dunaújvárosiak edzője összesség...",2.0


In [36]:

train.to_csv(os.path.join(save_dir, "train.csv"), index=False)

In [37]:
dev.to_csv(os.path.join(save_dir, "test.csv"), index=False)

In [38]:
test.to_csv(os.path.join(save_dir, "val.csv"), index=False)